In [1]:
# Making a function StockVol to calibrate stock volitility under geometric Brownian motion model
!pip install pandas_datareader
!pip install yfinance
!pip install cvxpy
import numpy as np
import matplotlib as m
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import data
import yfinance as yf
import datetime as dt
from datetime import date
yf.pdr_override()

/home/mayuresh/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [11]:
#USE    : To get historical data for a particular stock
#INPUT  : code of the company "goog"
#OUTPUT : array of close price of data
def GetData(companyName, years=1):
    # Getting today's date
    today = date.today()
    # Setting start date to historical years
    startDate = today.replace(today.year-years)
    # Date : Open High Low Close "Adj Close" Volume
    data = pdr.get_data_yahoo(companyName, startDate, today)
    # Cleaning to get the close prices only
    closePrice = [data["Close"][i] for i in range(len(data))]
    # Return close price of the data
    return closePrice

In [12]:
#USE        : to find the volitility of the stock data
#INPUT      : array of 1 year historical prices
#ASSUMPTION : Stock doesn't pay dividends
#OUTPUT     : historical volitility of the stock (standard deviation)
def StockVol(histoPrice):
    #Finding the ratio of current/previous values
    Sn = [np.log(histoPrice[i+1]/histoPrice[i]) for i in range(len(histoPrice)-1)]
    return np.sqrt(np.var(Sn, ddof=1))

In [13]:
#USE        : to generate n stock path
#INPUT      : nSteps-> number of time steps, sigma-> volatility, 
#           : T-> TerminalTime (yearly unit), nSimulations-> number of Simulations
#           : r-> interest rate, delta-> continuous dividend yield 
#           : S0-> initial price of the stock
#ASSUMPTION : Stock is not paying any dividends
#OUTPUT     : Stock Path as a matrix
def StockPath(nSteps, sigma = 0, S0=0, T=1, nSimulations=10, r=.02, delta=0):
    path = []
    for period in range(nSimulations):
        # Step size for every path simulation
        step =  T/nSteps
        #
        periodPrice = [S0]
        # Computation of n simulated stock prices 
        # np.random.normal(0,1,n) outputs array of nSteps normal values
        Y = np.exp(((r-delta-(sigma**2)/2)*step)+(sigma*np.random.normal(0,1,nSteps)*np.sqrt(step)))
        # now we have array of simulated factor for nSteps stock path 
        # we want to multiply this to previous to find simulated price
        counter = 0
        for y in Y:
            periodPrice.append(periodPrice[counter]*y)
            counter += 1
        path.append(periodPrice)
    return(path)      

In [14]:
#USE        : To generate the European put option price through Monte Carlo method
#INPUT      : StockPath, nSteps-> number of time steps, sigma-> volatility, 
#           : T-> TerminalTime (yearly unit), nSimulations-> number of Simulations
#           : r-> interest rate, delta-> continuous dividend yield,
#           : S0-> initial price of the stock, K-> Strike Price
#ASSUMPTION : Stock is not paying any dividends
#OUTPUTS    : Discounted Payoff Vector, Price & Variance
def EurOptPrice(StockPath, nStockPath, T=1, r=.02, K = 0):
    # Last Column of StockPath is Terminal Value
    Dis_Payoff_Vec = []
    for j in range(0,nStockPath):
        St_j = StockPath[j][nStockPath]
        # Create Column of Disounted Payoffs
        Dis_Payoff_j = np.exp(-r*T)*max(0,K - St_j)
        Dis_Payoff_Vec = np.append(Dis_Payoff_Vec,Dis_Payoff_j)
        Price = np.mean(Dis_Payoff_Vec)
        Price_Var = np.var(Dis_Payoff_Vec)
    return (Dis_Payoff_Vec, Price, Price_Var)

In [15]:
HistoPrice = GetData("goog", years=1)
sigma = StockVol(HistoPrice)
StockPaths = StockPath(nSteps = 20, sigma = sigma, S0 = HistoPrice[0], T = 1, nSimulations = 52, r = 0.02)
Eu = EurOptPrice(StockPaths, nStockPath = 20, T = 1, r = 0.02, K = 1200)

In [96]:
import cvxpy as cp
#USE        : To generate the Aemrican put option price through Monte Carlo method
#INPUT      : StockPaths, nSteps-> number of time steps, sigma-> volatility, 
#           : T-> TerminalTime (yearly unit), nSimulations-> number of Simulations
#           : r-> interest rate, delta-> continuous dividend yield
#           : S0-> initial price of the stock, K-> Strike Price
#ASSUMPTION : Stock is not paying any dividends
#OUTPUTS    : Discounted Payoff Vector, Price & Variance
def AmericanOptionPrice(StockPaths, nSteps, nSimulations, sigma, r, K, T, mcSamples=1000, delta = 0):
    step = T/nSteps
    # Shape is (nSimulations, nSteps)
    # Take a closer look, all stock path has been reversed to work in the backward direction
    # i.e. it starts from time in reverse direction
    paths = np.array(StockPaths).T[::-1] 
    #print(f"shape of paths is {np.shape(paths)}")
    
    # This contains payoffs of all the path or 0th time step
    exercisedPayoffs = [(K-paths[0]).clip(min=0)]
    exercisedExpDisPayoffs = [(K-paths[0]).clip(min=0)]
    
    for i in range(1,nSteps):
        # Has number of values equal to nSsimulations i.e. every path value
        # So basically variable 'paths' contain price at each time step for all simulation
        # stockPrices has price at time step i for every path of simulation
        stockPrices = paths[i]
        #print(f"stockPrices shape is {np.shape(stockPrices)}")
        
        # Make positive payoff using clip, this is payofsf at time step i for all path 
        payOffs = (K - stockPrices).clip(min=0)
        #print(payOffs, np.shape(payOffs))
        
        # We need to find Discounted Conditional Expected Price for next step of each step value
        # So we need to Simulate 1 step Monte Carlo to generate next time stock price
        # Find the payoff of this next time stock price and calculate average 
        # We use blackSholes model for price simulation
        
        # For every path in the stockPrices simulate Monte Carlo and get next prices using Black-Sholes
        # time step is ith
        simulatedPrices = [ stockPrice*np.exp(sigma*np.random.normal(0,1,mcSamples)*np.sqrt(step) + (r-(sigma**2)/2)*step) for stockPrice in stockPrices]
        simulatedPrices = np.array(simulatedPrices)
        #print(f"Shape of simulatedPrices is: {np.shape(simulatedPrices)}")
        # shape is nSimulations X mcSamples
        # So for every path at timeStep i we have mcSamples montecarlo samples simulated
        
        #If this is the t-1 step then the expected payoff is to be simulated mean
        if(i == 1):
            # Find expected payoff by taking mean of all the path
            expectedPayoffs = np.array([np.mean(expectedPayoff) for expectedPayoff in K-simulatedPrices.clip(min=0)])
            #print(f"Shape of expectedPayoffs is {np.shape(expectedPayoffs)}")
            discountedExpectedPayoffs = expectedPayoffs* np.exp(-r*step)
            #print(f"Shape of discountedExpectedPayoff is {np.shape(discountedExpectedPayoffs)}")
            
        # For other time step it must be done by machine learning
        else:
            # discountedExpectedPayoffs needs to be calculated using the machine learning
            # Take as input stockPrices (has stock price for each path at time step i)
            # and take another input as discountedExpectedPayoffs (has expectedPayoffs 
            # for each path at time step i)
            # run the machine learning model on this to get the weights
            # use this weights to predict new expected payoff at timestep i-1
            
            X = np.array([np.array([1]*len(stockPrices)), stockPrices, stockPrices**2])
            X = X.T
            #print(np.shape(X))
            #print(X)
            # Formalizing the regression model
            beta = cp.Variable(3)
            loss = cp.sum_squares(discountedExpectedPayoffs-X@beta)
            prob = cp.Problem(cp.Minimize(loss))
            prob.solve()
            # estimating discounted expected payoffs using estimated parameters
            discountedExpectedPayoffs = X@beta.value
        exercisedPayoffs.insert(0, payOffs)
        exercisedExpDisPayoffs.insert(0, discountedExpectedPayoffs)
    exercisedPayoffs = np.array(exercisedPayoffs).T
    exercisedExpDisPayoffs = np.array(exercisedExpDisPayoffs).T
    #Now we have nSimulations X nSteps shaped arrays
    discountedPayoffsMax = []
    for i in range(nSimulations):
        for j in range(nSteps):
            exerPayoff = exercisedPayoffs[i][j]
            exerDisPayoff = exercisedExpDisPayoffs[i][j]
            if exerPayoff > exerDisPayoff :
                disPayoff = exerPayoff*np.exp(-r*step*j)
                break
            else:
                disPayoff = exerDisPayoff.np.exp(-r*step*j)
        discountedPayoffsMax.append(disPayoff) 
        
    return(discountedPayoffsMax, np.mean(discountedPayoffsMax), np.var(discountedPayoffsMax, ddof=1))

In [97]:
HistoPrice = GetData("goog", years=1)
sigma = StockVol(HistoPrice)
nSteps = 20
S0 = HistoPrice[0]
StockPaths = StockPath(nSteps = 20, sigma = sigma, S0 = HistoPrice[0], T = 1, nSimulations = 52, r = 0.02)
mcSamples = 1000
nSimulations = 52
r = 0.04
K = 1200
T = 1

In [99]:
a,b,c = AmericanOptionPrice(StockPaths, nSteps, nSimulations, sigma, r, K, T, mcSamples=1000)

In [102]:
print(len(a))
print(b)
print(c)

52
145.07093052652087
27.12703697400504
